In [ ]:
import os
import random
import shutil
import pathlib
import json
import time
import warnings
import numpy as np
from IPython.core.debugger import set_trace
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
from torch.utils.data import Dataset
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import model_select

In [ ]:
cifar10_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

class trainArgs:
    """Argument class for training setup and parameters. Some advanced settings are hardcoded for simplicity."""
    
    arch = None
    world_size = -1
    rank = -1
    dist_url = 'tcp://224.66.41.62:23456'
    dist_backend = 'ncc1'
    multiprocessing_distributed = False
    
    def __init__(self, root, workers=4, epochs=90, start_epoch=0, batch_size=256, lr=0.1, lrdecay=0.1,
                 momentum=0.9, weight_decay=1e-4, print_freq=10, resume='', main_file='candidate.npz',
                 sub_file=None, test_file='candidate.npz', outpath='./results/unknown/train_logs.txt',
                 model='resnet', inject_noise=0, mix_cifar=False, mix_rate=0.04, 
                 evaluate=False, track_correct=False, pretrained=False, seed=None, gpu=None):
        """
        Args:
            root (pathlib.Path): Directory containing the npz files.
            main_file (npz filename): npz file that contains the training set in the 'X_train' field.
            test_file (npz filename, optional): npz file that contains the training set in the 'X_test' field.
                This is used in case the main file is for the CIFAR-10 dataset.
            sub_file (npz filename, optional): npz file that contains the CIFAR-10 dataset.
                If given, this is used for tracking the performance on a second test set.
            outpath (pathlib.Path): Output log file for accuracy results.
            inject_noise (optional, TODO): Symmetric noise level per class for injecting noise to CIFAR-10 training set.
            mix_cifar (boolean, optional, TODO): Replace fraction of one training set with random images from another.
            mix_rate (conditional, TODO): Determines the fraction when mix_cifar is set to True.
            evaluate (boolean, optional): Evaluate the performance on the test set without training.
            track_correct (boolean): Record the indices of correctly classified test images and write it to
                a file with '_corr' added before the extension to the outpath filename.
            gpu: Cuda device for training.
            
            train (boolean): Extract train (True) or test (False) set from the file.
        """
        self.root = root
        self.workers = workers
        self.epochs = int(epochs)
        self.start_epoch = start_epoch
        self.batch_size = batch_size
        self.lr = lr
        self.lrdecay = lrdecay
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.print_freq = print_freq
        self.resume = resume
        self.main_file = main_file
        self.sub_file = sub_file
        self.test_file = test_file
        self.outpath = outpath
        self.model = model
        self.inject_noise = inject_noise
        self.mix_cifar = mix_cifar
        self.mix_rate = mix_rate
        self.evaluate = evaluate
        self.track_correct = track_correct
        self.pretrained = pretrained
        self.seed = seed
        self.gpu = gpu

best_acc1 = 0

class CandidateDataset(Dataset):
    """Candidate dataset."""
    
    def __init__(self, pathname, transform=None, train=True):
        """
        Args:
            pathname (pathlib.Path): Path to the npz file.
            transform (callable, optional): Optional transform to be applied on a sample.
            train (boolean): Extract train (True) or test (False) set from the file.
        """
        self.samples, self.targets = np_loader(pathname.resolve(), train=train)
        self.transform = transform
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """
        sample, target = self.samples[index], self.targets[index]
        sample = Image.fromarray(np.moveaxis(sample, 0, -1))
        
        if self.transform is not None:
            sample = self.transform(sample)
            
        # TODO: Target transform.
        
        return sample, target
    
def np_loader(filename, train=True):
    data = np.load(filename)
    if train:
        samples = data['X_train'].transpose(0, 3, 1, 2)
        targets = data['y_train']
    else:
        samples = data['X_test'].transpose(0, 3, 1, 2)
        targets = data['y_test']
    return samples, targets

def main_worker(gpu, ngpus_per_node, args):
    global best_acc1

    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))

    if args.distributed:
        if args.dist_url == "env://" and args.rank == -1:
            args.rank = int(os.environ["RANK"])
        if args.multiprocessing_distributed:
            # For multiprocessing distributed training, rank needs to be the
            # global rank among all the processes
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                world_size=args.world_size, rank=args.rank)
    # create model
    if args.pretrained:
        print("=> using pre-trained model '{}'".format(args.arch))
        model = models.__dict__[args.arch](pretrained=True)
    else:
        print("=> creating model '{}'".format(args.model))
        model = model_select.BaseModel.create(args.model)
    
    if args.distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if args.gpu is not None:
            torch.cuda.set_device(args.gpu)
            model.cuda(args.gpu)
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int(args.workers / ngpus_per_node)
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        else:
            model.cuda()
            # DistributedDataParallel will divide and allocate batch_size to all
            # available GPUs if device_ids are not set
            model = torch.nn.parallel.DistributedDataParallel(model)
    elif args.gpu is not None:
        torch.cuda.set_device(args.gpu)
        model = model.cuda(args.gpu)
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        if args.arch.startswith('alexnet') or args.arch.startswith('vgg'):
            model.features = torch.nn.DataParallel(model.features)
            model.cuda()
        else:
            model = torch.nn.DataParallel(model).cuda()

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda(args.gpu)

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            best_acc1 = checkpoint['best_acc1']
            if args.gpu is not None:
                # best_acc1 may be from a checkpoint from a different GPU
                best_acc1 = best_acc1.to(args.gpu)
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    main_file = args.root / args.main_file
    if args.sub_file:
        sub_file = args.root / args.sub_file
    normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                     std=[0.2023, 0.1994, 0.2010])

    train_dataset = CandidateDataset(
        main_file,
        transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
        
    # TODO: Inject symmetric noise to CIFAR-10 training set
    if args.inject_noise:
        im_per_class = int(len(train_dataset) / len(cifar10_classes))
        noisy_idx = []
        num_shuffle = int(im_per_class * args.inject_noise)
        for i in range(len(cifar10_classes)):
            cur_idx = [idx for idx, label in enumerate(train_dataset.targets) if label==i]
            cur_idx = random.sample(cur_idx, len(cur_idx))
            for r in range(len(cifar10_classes)):
                noisy_idx += [r for idx in cur_idx[im_per_class - (r+1)*num_shuffle:im_per_class - r*num_shuffle]]
            noisy_idx += [i for idx in cur_idx[:im_per_class - len(cifar10_classes)*num_shuffle]]
        train_dataset.targets = noisy_idx
    
    # TODO: Replace fraction of one training set randomly with another.
    if args.mix_cifar:
        assert args.mix_rate, "mix_rate should be given when mix_cifar is set"
        assert args.traindir2, "traindir2 must be given when mix_cifar is set"
        assert not args.inject_noise, "inject_noise should not be given when mix_cifar is set"
        assert not args.testdir2, "only one testdir can be set when mix_cifar is set"
        
        traindir2 = os.path.join(args.root, args.traindir2)
        clean_dataset = datasets.ImageFolder(
            traindir2,
            transforms.Compose([
                transforms.ToTensor(),
                normalize,
            ]))
        
        im_per_class = int(len(train_dataset) / len(train_dataset.classes))
        num_shuffle = int(im_per_class * args.mix_rate)
        shuffled_samples = []
        clean_samples = []
        for i in range(len(train_dataset.classes)):
            cur_imgs = [s[0] for s in train_dataset.samples if s[1]==i]
            cur_imgs = random.sample(cur_imgs, im_per_class - num_shuffle)
            mix_imgs = [s[0] for s in clean_dataset.samples if s[1]==i]
            mix_imgs = random.sample(mix_imgs, num_shuffle)
            clean_samples += [(img, i) for img in mix_imgs]
            shuffled_samples += [(img, i) for img in cur_imgs + mix_imgs]
            
        train_dataset.samples = shuffled_samples
        clean_dataset.samples = clean_samples
        
        val_loader2 = torch.utils.data.DataLoader(
            clean_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None),
            num_workers=args.workers, pin_memory=True, sampler=train_sampler)
        
    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None),
        num_workers=args.workers, pin_memory=True, sampler=train_sampler)

    val_loader = torch.utils.data.DataLoader(
        CandidateDataset(main_file, transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]), train=False),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True)
    
    if args.sub_file:
        val_loader2 = torch.utils.data.DataLoader(
            CandidateDataset(sub_file, transforms.Compose([
                transforms.ToTensor(),
                normalize,
            ]), train=False),
            batch_size=args.batch_size, shuffle=False,
            num_workers=args.workers, pin_memory=True)

    if args.evaluate:
        validate(val_loader, model, criterion, args)
        return

    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            train_sampler.set_epoch(epoch)
        if epoch < 70:
            adjust_learning_rate(optimizer, epoch, args)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, args)
        
        with open(args.outpath, 'a') as fn:
            print('Epoch {}'.format(epoch), file = fn)

        # evaluate on validation set
        dum_acc = validate(train_loader, model, criterion, args)
        acc1 = validate(val_loader, model, criterion, args)
        if args.sub_file or args.mix_cifar:
            dum_acc = validate(val_loader2, model, criterion, args)

        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        if not args.multiprocessing_distributed or (args.multiprocessing_distributed
                and args.rank % ngpus_per_node == 0):
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': args.arch,
                'state_dict': model.state_dict(),
                'best_acc1': best_acc1,
                'optimizer' : optimizer.state_dict(),
            }, is_best)


def train(train_loader, model, criterion, optimizer, epoch, args):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            input = input.cuda(args.gpu, non_blocking=True)
        target = target.cuda(args.gpu, non_blocking=True)
        
        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(acc1[0], input.size(0))
        top5.update(acc5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Acc@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))


def validate(val_loader, model, criterion, args):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    if args.track_correct:
        corr_dict = {'correct':[]}
    
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            if args.gpu is not None:
                input = input.cuda(args.gpu, non_blocking=True)
            target = target.cuda(args.gpu, non_blocking=True)

            # compute output
            output = model(input)
            loss = criterion(output, target)
            
            # record correctly classified examples
            if args.track_correct:
                correct = accuracy(output, target, topk=(1, 5), track=True)
                corr_dict['correct'] += [(i*args.batch_size) + idx for idx, is_corr in 
                                         enumerate(correct) if is_corr]

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), input.size(0))
            top1.update(acc1[0], input.size(0))
            top5.update(acc5[0], input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                      'Acc@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                       i, len(val_loader), batch_time=batch_time, loss=losses,
                       top1=top1, top5=top5))

        # Record the indices of the correctly classified images
        if args.track_correct:
            fname, ext = str(args.outpath).split('.')
            corrfile = fname + '_corr.json'
            with open(corrfile, 'w') as f:
                json.dump(corr_dict, f, indent=2)
            return
        
        with open(args.outpath, 'a') as fn:
            print('{top1.avg:.3f}, {top5.avg:.3f}'
                  .format(top1=top1, top5=top5), file = fn)

    return top1.avg


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch, args):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (args.lrdecay ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,), track=False):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        
        # return indices of the correctly classified examples instead of accuracy.
        if track:
            return correct[:1].view(-1).cpu().numpy()

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

### Setup and parameter selection

In [ ]:
# set parameters

# directories
root = pathlib.Path.cwd()
main_file = 'candidate.npz'
sub_file = 'cifar.npz'

# TODO: Experiments with synthetic noise and fractions
# mix_cifar = True
# inject_noise = 0.05

# model and output
model = 'resnet'
outpath = pathlib.Path.cwd() / 'results' / model / 'train_logs.txt'

# evaluate mode with recording indices
# evaluate = True
# track_correct=True
# resume_file = 'model_best.pth.tar'

# run parameters
epochs = 50
print_freq = 1000
gpu = 0
batch_size = 128 if model in ['densenet', 'pyramidnet', 'resnet_basic'] else 256

In [ ]:

args = trainArgs(root, epochs=epochs, batch_size=batch_size, print_freq=print_freq, outpath=outpath,
                 main_file=main_file,
                 sub_file=sub_file, 
                 # evaluate=evaluate, 
                 # track_correct=track_correct, 
                 # resume=resume_file, 
                 model=model, gpu=gpu)

args.outpath.parent.mkdir(exist_ok=True, parents=True)

if args.seed is not None:
    random.seed(args.seed)
    torch.manual_seed(args.seed)
    cudnn.deterministic = True
    warnings.warn('You have chosen to seed training. '
                  'This will turn on the CUDNN deterministic setting, '
                  'which can slow down your training considerably! '
                  'You may see unexpected behavior when restarting '
                  'from checkpoints.')

if args.gpu is not None:
    warnings.warn('You have chosen a specific GPU. This will completely '
                  'disable data parallelism.')

if args.dist_url == "env://" and args.world_size == -1:
    args.world_size = int(os.environ["WORLD_SIZE"])

args.distributed = args.world_size > 1 or args.multiprocessing_distributed

ngpus_per_node = torch.cuda.device_count()
if args.multiprocessing_distributed:
    # Since we have ngpus_per_node processes per node, the total world_size
    # needs to be adjusted accordingly
    args.world_size = ngpus_per_node * args.world_size
    # Use torch.multiprocessing.spawn to launch distributed processes: the
    # main_worker process function
    mp.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args))
else:
    # Simply call main_worker function
    main_worker(args.gpu, ngpus_per_node, args)

#### Dataset test

```
def get_item_test():
    normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                         std=[0.2023, 0.1994, 0.2010])
    set_trace()
    
    train_dataset = CandidateDataset(
            root / main_file,
            transforms.Compose([
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ]))
    
    train_dataset2 = datasets.ImageFolder(
        '/root/dockspace/cifar/tiny_train/',
        transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
    
    test_sample = train_dataset[0]
    test_sample2 = train_dataset2[0]
    i = 1
    return test_sample
```

`get_item_test()`